# Policy Gradient Methods

## Introduction

In this chapter we consider methods that instead learn a
parameterized policy that can select actions without consulting a value function. A value
function may still be used to learn the policy parameter, but is not required for action
selection. We will use the notation $\theta \in R^{d^\prime}$ for the policy's parameter vector. Thus we write $\pi(a | s, \theta) = P(A_t = a | S_t=s, \theta_t = \theta)$ for the probability that action a is taken at time t given state s and current parameter $\theta$.
If a method uses a learned value function as well, the nthe value function's weight vector is denoted $w \in R^d$ as usual, as in $\hat{v} (s, w)$.

In this chapter we consider methods for learning the policy parameter based on the gradient of some scalar performance measure $J(\theta)$ with respect to the policy parameter. These methods seek to maximize performance, so their updates approximate gradient ascent in J:

$\theta_{t+1} = \theta_t + \alpha \hat{\nabla  J(\theta_t)} $

Where $\hat{\nabla  J(\theta_t)} \in R^{d^{\prime}}$ is a stochastic estimate whose expectation approximates the gradient of the performance measure with respect to its argument $\theta_t$ (ie. gradient of a batch). All methods that follow this general schema we call policy gradient methods, whether or not they also learn an approximate value function. Methods that learn approximations to both policy and value functions are often called
actor-critic methods, where 'actor' is a reference to the learned policy, and 'critic' refers to the learned value function, usually a state-value function. First we treat the episodic case, in which **performance is defined as the value of the start state under the parameterized policy**, before going on to consider the continuing case, in which performance is defined as the average reward rate.

## Policy Approximation and its Advantages

In policy gradient methods, the policy can be parameterized in any way, as long as $\pi(a | s, \theta)$ is differentiable with respect to its parameters, that is, as long as $\nabla \pi(a | s, \theta)$ (ie. the column vector of partial derivatives of $\pi(a | s, \theta)$) with respect to the components of $\theta$ exists and is finite for all $s \in S, a \in A(s), \theta \in R^{d^\prime}$. in practice, **to ensure exploration we generally require that the policy never becomes deterministic**. In this section we introduce the most common parameterization for discrete action spaces and point out the advantages it offers over action-value methods. Policy-based methods also offer useful ways of dealing with continuous action spaces.

If the action space is discrete and not too large, then a natural and common kind of parameterization is to form parameterized numerical preference $h(s, a, \theta) in R$ for each state-action pair. The actions with the highest preferences in each state are given the highest probabilities of being selected, for example, according to an exponential soft-max distribution:

$\pi (a | s, \theta) = \frac{e^{h(s, a | \theta)}}{\sum_{b} e^{h(s, b | \theta)}}$

Note that the denominator here is just what is required so that the action probabilities in each state sum to one. We call this kind of policy parameterization **soft-max in action preferences**.

The action preferences themselves can be parameterized arbitrarily (ie $h(s, a | \theta)$). For example, they might be computed by a DNN, where $\theta$ is the vector of all the connection weights of the network. Or the preferences could simply be linear in features,

$h(s, b | \theta) = \theta^T \phi(s, a)$

One advantage of parameterizing policies according to the soft-max in action preferences is that the approximate policy can approach a deterministic policy, whereas with $\epsilon-$greedy, action selection over action values there is always an $\epsilon$ probability of selecting a random action. Of course, one could select according to a soft-max distribution based on action values, but this alone would not allow the policy to approach a deterministic policy. Instead, the action-value estimates woudl converge to ehir corresponding true values, which would differ by a finite amount, translating to specific probabilities other than 0 and 1. If the soft-max distribution included a temperature parameter, then the temperature could be reduced over time to approach determinism, but in practice it would be difficult to choose
the reduction schedule, or even the initial temperature, without more prior knowledge of the true action values than we would like to assume. Acton preferences are different because they do not approach specific values; instead they are driven to produce the optimal stochastic policy. If the optimal policy is terministic, then the preferences of the optimal actions will be driven infinitely higher than all suboptimal actions (if permitted by the parameterization).

A second advantage of parameterizing policies according to the soft-max in action preferences is that it enables the selection of actions with arbitrary probabilities. In problems with significant function approximation, the best approximate policy may be stochastic. For example, in card games with imperfect information the optimal play is often to do two different things with specific probabilities such as when bluffing in Poker. **Action-value methods have no natural way of finding stochastic optimal policies, whereas policy approximating methods can**.

Perhaps the simplest advantage that policy parameterization may have over action-value parameterization is that the policy may be a simpler function to approximate. Problems vary in the complexity of their policies and action-value functions. For some,
the action-value function is simpler and thus easier to approximate. For others, the policy
is simpler. In the latter case a policy-based method will typically learn faster and yield a
superior asymptotic policy.

Finally, we note that the choice of policy parameterization is sometimes a good way
of injecting prior knowledge about the desired form of the policy into the reinforcement
learning system. This is often the most important reason for using a policy-based learning
method.

## The Policy Gradient Theorem

In addition to the practical advantages of policy parameterization over $\epsilon-$greedy action selection, there is also an important theoretical advantage. With continuous policy parameterization the action probabilities change smoothly as a function of the learned parameter, whereas in $\epsilon-$greedy selection the action probabilities may change dramatically for an arbitrarily small change in the estimated action values, if that change results in a different action having the maximal value. Largely because of this, stronger convergence guarantees are available for
policy-gradient methods than for action-value methods. In particular, it is the continuity of the policy dependence on the parameters that enables policy-gradient methods to approximate gradient ascent.

The episodic and continuing cases define the performance measure, $J(\theta)$, differently and thus have to be treated separately to some extent. Nevertheless, we will try to present both cases uniformly, and we develop a notation so that the major theoretical results can be described with a single set of equations.

In this section, we treat the episodic case, for which we define the performance measure as the value of the start state of the episode. We can simplify the notation without losing any meaningful generality by **assuming that every episode starts in some particular (non-random) state $s_0$**. Then, in the episodic case, we define performance as

$J(\theta) = v_{\pi_{\theta}} (s_0)$

Where $v_{\pi_{\theta}}$ is the true value function for $\pi_{\theta}$, the policy determined by $\theta$. From here on in our discussion, we will assume no discounting ($\gamma = 1$) for the episodic case, although for completeness, we do include the possibility of discounting in the boxed algorithm.

With function approximation, it may seem challenging to change the policy parameter in a way that ensures improvement. The problem is that performance depends on both the action selections, and the distribution of states in which those selections are made, and that both of these are affected by the policy parameter. Given a state, the effect of the policy parameter on the actions, and thus on reward, can be computed in a relatively straightforward way from knowledge of the parameterization. But the effect of the policy on the state distribution ($\mu$) is a function of the environment and is typically unknown.
How can we estimate the performance gradient with  respect to the policy parameter when the gradient depends on the unkonwn effect of policy changes on the state distribution?

Fortunately, there is an excellent theoretical answer to this challenge in the form of policy gradient theorem, which provides an analytic expression for the gradient of performance with respect to the policy parameter (which is what we need to approximate for gradient ascent), that does not involve the derivative of the state distribution. The policy gradient theorem for the episodic case establishes that

$\nabla J(\theta) \propto \sum_{s} \mu (s) \sum_a q_{\pi} (s, a) \nabla \pi(a | s, \theta)$

Where the gradients are column vectors of partial derivatives w.r.t the components of $\theta$, and $\pi$ denotes the policy corresponding to parameter vector $\theta$. In the episodic case, the constant of proportionality is the average length of an episode, and in the continuing case it is 1, so that the relationship is actually an equality. The distribution $\mu$ here is the on-policy distribution under $\pi$ (the state distribution).

<img src='pngs/on-policy distribution.png'>
<img src='pngs/on-policy distribution 2.png'>
<img src='pngs/proof_PGT_1.png'>

Let $\eta (s) = \sum_{k=0}^{\infty} P^{k}_{\pi}(s | s_0)$ (This is the expected visits to state s given we start at $s_0$, since we always start in state $S_0$, this is exactly as the on-policy distribution)

Then

$\nabla J(\theta) $

$= \nabla v_{\pi} (s_0) $

$= \sum_{s} \eta (s) \sum_{a} \nabla \pi(a | s) q_{\pi} (s, a) $

$= \sum_{s^\prime} \eta (s^\prime) \sum_{s} \frac{\eta (s)}{\sum_{s^{\prime}} \eta (s^{\prime})} \sum_{a} \nabla \pi(a | s) q_{\pi} (s, a)$

$= \sum_{s^\prime} \eta (s^\prime) \sum_{s} \mu (s) \sum_{a} \nabla \pi(a | s, \theta) q_{\pi} (s, a)$

$\propto  \sum_{s} \mu (s) \sum_{a} \nabla \pi(a | s) q_{\pi} (s, a)$

## REINFORCE: Monte Carlo Policy Gradient

We are now ready to derive our first policy-gradient learning algorithm. Recall our overall strategy of stochastic gradient ascent, which requires a way to obtain samples such that the xpectation of the sample gradient is proportional to the actual gradient of the performance measure as a function of the parameter. The sample gradients need only to be
proportional to the gradient because any constant proportionality can be sborbed into the step size $\alpha$, which is otherwise arbitrary. The policy gradient theorem gives an exact expression proportional to the gradient; all that is needed is some way of sampling whose expectation equals or approximates this expression. Notice that the right-hand side of the policy gradient theorem is a sume over states weighted by how often the states occur under the target policy $\pi$; if $\pi$ is followed, then states will be
encountered in these proportions. Thus:

$\nabla J(\theta) \propto  \sum_{s} \mu (s) \sum_{a} \nabla \pi(a | s) q_{\pi} (s, a) = E^{\pi}_{S_t \sim \mu} [\sum_{a} \nabla \pi(a | S_t, \theta) q_{\pi} (S_t, a)]$

We could stop here and instaniate our stochastic gradient ascent update algorithm as:

$\theta_{t+1} = \theta_{t} + \alpha \sum_{a} \hat{q} (S_t, a, w) \nabla \pi(a| S_{t}, \theta)$

Where $\hat q$ is some learned approximation to $q_{\pi}$. This algorithm, which has been called all-actions method because its update involves all of the actions, is promising and deserving of further study, but our current interest is the classical REINFORCE algorithm, whose update at time t involves just $A_t$, the one action actually taken at time t.

We continue our derivation of REINFORCE by introducing $A_t$ in the same way as we introduced $S_t$ by replacing the sum over the random variable's possible values by an expectation under $\pi$, then sampling the expectation. The previous equation involves an appropriate sum over actions, but each term is not weighted by action probability ($\pi$), so it is not a expectation under $\pi$. So we introduce such a weighting, without changing the equality, by multiplying and ten dividing the summed terms by $\pi(a | S_t, \theta)$. We have:

$\nabla J(\theta) \propto  E^{\pi}_{ S_t \sim \mu} [\sum_{a} \nabla \pi(a | S_t, \theta) q_{\pi} (S_t, a)]$

$= E^{\pi}_{S_t \sim \mu} [\sum_{a} \nabla \pi(a | S_t, \theta) q_{\pi} (S_t, a)  \frac{\pi(a | S_t, \theta)}{ \pi(a | S_t, \theta)}]$

$= E^{\pi}_{S_t \sim \mu} [E_{A_t \sim \pi}[\nabla \pi(A_t | S_t, \theta) q_{\pi} (S_t, A_t)  \frac{1}{ \pi(A_t | S_t, \theta)}]]$

$= E^{\pi}_{S_t \sim \mu, A_t \sim \pi} [q_{\pi} (S_t, A_t)  \frac{\nabla \pi(A_t | S_t, \theta)}{ \pi(A_t | S_t, \theta)}]$

Since $E^{\pi}[G_t | S_t, A_t] = q_{\pi} (S_t, A_t)$, $G_t (S_t, A_t)$ is an unbiased estimator of $q_{\pi} (S_t, A_t)$ given $S_t, A_t$, that is, we can represent a sample

$G_t(S_t, A_t) \frac{\nabla \pi(A_t | S_t, \theta)}{ \pi(A_t | S_t, \theta)}$

Where $G_t (S_t, A_t)$ is the return as usual given current $S_t, A_t$. The final expression in brackets is exactly what is needed, a quantity that can be sampled on each time step whose expectation is proportional to the gradient. Using this sample to instantiate our generic Stochastic Ascent algorithm yields the REINFORCE update:

$\theta_{t+1} = \theta_{t} + \alpha G_{t} (S_t, A_t) \frac{\nabla \pi(A_t | S_t, \theta)}{ \pi(A_t | S_t, \theta)}$

This update has an intuitive appeal. Each increment is proportional to the product of a return $G_t (S_t, A_t)$ and a vector, the gradient of the probability of taking the action actually taken divided by the probability of taking that action. The vector is the direction in parameter space that most increases the probability of repeating the action $A_t$ on future visits to state $S_t$.
The update increases the parameter vector in this direction proportional to the return, and inversely proportional to the action probability. The former makes sense because it causes the parameter to move most in the directions that favor actions that yield the highest return. THe latter makes sense because otherwise actions that are selected frequently are at an advantage (the updates will be more often in their direction)
and might win out even if they do not yield the highest return). Note that REINFORCE uses the complete return from time t, which includes all
future rewards up until the end of the episode. In this sense REINFORCE is a Monte
Carlo algorithm and is well defined only for the episodic case with all updates made in
retrospect after the episode is completed.

Notice that $\nabla ln \pi(A_t | S_t, \theta) = \frac{\nabla \pi(A_t | S_t, \theta)}{ \pi(A_t | S_t, \theta)}$ which is refer as eligibility vector.

<img src='pngs/REINFORCE.png'>

The second difference between the pseudocode update, and the REINFORCE update
equation (13.8) is that the former includes a factor of $\gamma^t$, this is because as mentioned earlier, in the text we are treating the non-discounted case while in the algorithm above we are giving the algorithms for the general discounted case. ll of the ideas
go through in the discounted case with appropriate adjustments but involve additional complexity that distracts from the main ideas.

As a stochastic gradient method, REINFORCE has good theoretical convergence
properties. By construction, the expected update over an episode is in the same direction
as the performance gradient. This assures an improvement in expected performance for
sufficiently small $\alpha$, and convergence to a local optimum under standard stochastic approximation
conditions for decreasing $\alpha$. However, as a Monte Carlo method REINFORCE
may be of high variance and thus produce slow learning.